***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/diabetes_prediction_dataset.csv')
df = df.dropna()

encoder = OrdinalEncoder()
df['gender'] = encoder.fit_transform(df[['gender']])
df['smoking_history'] = encoder.fit_transform(df[['smoking_history']])
X = df.loc[:, df.columns != 'diabetes']
y = df['diabetes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)

cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

Best Parameters: {'C': 5}
Best Score: 0.8368286465493654
Best Estimator: LinearSVC(C=5, random_state=42)


In [ ]:
best = LinearSVC(C=5, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')

print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.80913955 0.83713294 0.75950822 0.6521099  0.84543515 0.78706052
 0.84074084 0.79150013 0.82981491 0.85196541]
Mean CV Accuracy: 0.8004407563924187
Training Set Accuracy: 0.94369


##**Extracting weights and intercept values**

In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['diabetes'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Diabetes_Predicted': predicted_label})
index_class = results_df.index[results_df['Diabetes_Predicted'] == 1]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

2897


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.6 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=01659d5eeda4c654cccaea0844450f0d1bbf4b639e4fabe5eeddb59ed2963b51
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.0 MB/s eta 0:00:00


##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    # Gender
    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = og_x_np[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    original_value2 = og_x_np[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    # heart disease
    original_value3 = og_x_np[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    #smoking status
    smoking = Or(X[4] == 3, X[4] == 4)
    opt.add(smoking)

    #bmi
    constraint5 = And(X[5] >= 18.5, X[5] <= 25)
    opt.add(constraint5)

    #hemoglobin
    original_value6 = og_x_np[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    # #glucose level
    constraint7 = And(X[7] >= 70, X[7] <= 127)
    opt.add(constraint7)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B < 0
    opt.add(inequality)
    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(l1_distance).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1.csv"


for i in range(len(filtered_records[:100])):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)
    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]
optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    # Gender
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    # heart disease
    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    #smoking status
    smoking = Or(X[4] == 3, X[4] == 4)
    opt.add(smoking)

    #bmi
    slack_bmi = Real('slack_bmi')
    slack_bmi_constr = (slack_bmi >= 0)
    constraint5 = And(X[5] > 0, X[5] >= 18.5 - slack_bmi, X[5] <= 25 + slack_bmi)
    opt.add(constraint5, slack_bmi_constr)

    #hemoglobin
    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    #glucose level
    slack_glucose = Real('slack_glucose')
    slack_glucose_constr = (slack_glucose >= 0)
    constraint7 = And(X[7] > 0, X[7] >= 70 - slack_glucose, X[7] <= 127 + slack_glucose)
    opt.add(constraint7, slack_glucose_constr)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])
    inequality = dot_product + B < 0
    opt.add(inequality)

    objective =  (slack_glucose + slack_bmi)
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_glucose = model[slack_glucose].as_decimal(15)
        print(f"slack_glucose value: {slack_glucose}")

        slack_bmi = model[slack_bmi].as_decimal(15)
        print(f"slack_bmi value: {slack_bmi}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_bmi, slack_glucose

    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()
    original_f, x_prime_optimized, cost, cost_optimized, slack_bmi, slack_glucose = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_bmi] + [slack_glucose]
        writer.writerow(data_row)

Optimized x': ['1', '73', '0', '0', '4', '0.385531905159113?', '9', '70']
Optimized distance: 18.114468094840886?
Optimized cost: 18.114468094840886?
slack_glucose value: 0
slack_bmi value: 18.114468094840886?
Optimized x': ['1', '80', '0', '0', '4', '0.335013760314923?', '9', '70']
Optimized distance: 18.164986239685076?
Optimized cost: 18.164986239685076?
slack_glucose value: 0
slack_bmi value: 18.164986239685076?
Optimized x': ['0', '80', '0', '0', '4', '0.395694316417773?', '9', '70']
Optimized distance: 18.104305683582226?
Optimized cost: 18.104305683582226?
slack_glucose value: 0
slack_bmi value: 18.104305683582226?
Optimized x': ['1', '59', '1', '0', '4', '0.287114243699851?', '9', '70']
Optimized distance: 18.212885756300148?
Optimized cost: 18.212885756300148?
slack_glucose value: 0
slack_bmi value: 18.212885756300148?
Optimized x': ['0', '62', '1', '1', '4', '0.215999189207618?', '8.8', '70']
Optimized distance: 18.284000810792381?
Optimized cost: 18.284000810792381?
slack_gl

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_bmi', 'Slack_glucose']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

      # Gender
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    # heart disease
    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    #smoking status
    smoking = Or(X[4] == 3, X[4] == 4)
    opt.add(smoking)

    #bmi
    constraint5 = And(X[5] >= 18.5 - slack_value1, X[5] <= 25 + slack_value1)
    opt.add(constraint5)

    #hemoglobin
    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    #glucose level
    constraint7 = And(X[7] >= 70 - slack_value2, X[7] <= 127 + slack_value2)
    opt.add(constraint7)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])
    inequality = dot_product + B < 0
    opt.add(inequality)

    # Calculate the L1 distance between og_x and X
    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])
    objective = l0
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Distance: {l1_distance_1}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_bmi'].iloc[i]
    slack_value2 = last['Slack_glucose'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2)
    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

Optimized x': ['1', '73', '0', '0', '3', '0.385531905159112?', '9', '70']
Distance: 115.524468094840887?
Objective: 115.524468094840887?
Optimized x': ['1', '80', '0', '0', '4', '1.743171673658583?', '9', '127']
Distance: 48.316828326341416?
Objective: 48.316828326341416?
Optimized x': ['0', '80', '0', '0', '4', '4.293931164680137?', '9', '127']
Distance: 35.446068835319862?
Objective: 35.446068835319862?
Optimized x': ['1', '59', '1', '0', '3', '0.287114243699850?', '9', '70']
Distance: 96.652885756300149?
Objective: 96.652885756300149?
Optimized x': ['0', '62', '1', '1', '4', '0.215999189207618?', '8.8', '70']
Distance: 252.944000810792381?
Objective: 252.944000810792381?
Optimized x': ['0', '54', '1', '0', '4', '4.012142530043487?', '9', '126.002210218218078?']
Distance: 18.470067688174590?
Objective: 18.470067688174590?
Optimized x': ['1', '65', '1', '0', '3', '0.243812976690549?', '9', '70']
Distance: 251.816187023309450?
Objective: 251.816187023309450?
Optimized x': ['1', '51', '

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0  & l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l0/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step1_optimized.csv')

d = pd.concat([d1, d3_initial])


l1_runtime = d.loc[:, 'Runtime'].mean()
print("L1 runtime", l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

0.5177770107007249


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Diabetes_l1/Step2.csv')
number_of_slacks_per_record = 2

count_s1 = 0
count_s2 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_glucose'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_glucose'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_bmi'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_bmi'].iloc[i] == 0:
        count_s2 = 0

    total_count = count_s1 + count_s2
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)


50.0
